In [ ]:
import time

class State:
    def __init__(self, board=None, turn=1):
        # 돌의 배치
        self.board = board if board is not None else [0]*9
        self.turn = turn  # -1이면 컴퓨터가 먼저, 1이면 사용자가 먼저

    def piece_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        return count

    def is_win(self):
        for line in [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                     (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                     (0, 4, 8), (2, 4, 6)]:            # Diagonals
            if all(self.board[i] == 1 for i in line):
                return True
        return False

    def is_lose(self):
        for line in [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # Rows
                     (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Columns
                     (0, 4, 8), (2, 4, 6)]:            # Diagonals
            if all(self.board[i] == -1 for i in line):
                return True
        return False

    def is_draw(self):
        return all(cell != 0 for cell in self.board) and not self.is_win() and not self.is_lose()

    def is_done(self):
        return self.is_win() or self.is_lose() or self.is_draw()

    def next(self, action):
        new_board = self.board[:]
        new_board[action] = 1 if self.turn == 1 else -1
        # 턴을 변경하여 새 상태 반환
        return State(new_board, -self.turn)

    def legal_actions(self):
        return [i for i in range(9) if self.board[i] == 0]

    def is_first_player(self):
        return self.turn == 1  # 사용자가 먼저인 경우

    def render(self):
        symbols = {1: 'o', -1: 'x', 0: '-'}
        for i in range(9):
            print(symbols[self.board[i]], end='')
            if i % 3 == 2:
                print()

In [ ]:
board = [0,1,1,
         0,1,-1,
         1,1,1]
st = State(board)
st.render()

-oo
-ox
ooo


In [ ]:
class Alphabeta:
    def __init__(self, state, depth_limit=3):
        self.state = state
        self.depth_limit = depth_limit # 최대 탐색 깊이 설정

    def evaluate_state(self, state):
        score = 0

        # 각 라인별로 AI와 상대의 돌을 셈
        for line in [(0, 1, 2), (3, 4, 5), (6, 7, 8),  # 행
                     (0, 3, 6), (1, 4, 7), (2, 5, 8),  # 열
                     (0, 4, 8), (2, 4, 6)]:            # 대각
            player_count = sum(1 for i in line if state.board[i] == 1)
            opponent_count = sum(1 for i in line if state.board[i] == -1)

            # AI가 승리할 수 있는 라인
            if player_count == 2 and opponent_count == 0:
                score += 1
            # 상대가 승리할 수 있는 라인
            elif opponent_count == 2 and player_count == 0:
                score -= 1

        return score

    def sorted_legal_actions(self, state, max_player): # 합법적인 행동 평가
        actions = state.legal_actions() # 가능한 모든 합법적인 행동 가져옴
        return sorted(
            actions, # 행동 정렬
            key=lambda action: self.evaluate_state(state.next(action)), # 각 행동에 대한 평가값
            reverse=max_player # max_player가 True면 내림차순(최대화), False면 오름차순(최소화)
        )

    def alpha_beta(self, state, depth, alpha, beta, max_player):
        if depth >= self.depth_limit or state.is_done():
            if state.is_done():
                return 1 # 승리
            if state.is_lose():
                return -1 # 패배
            return 0 # 무승부

        if max_player:
            max_eval = -float('inf')
            for action in self.sorted_legal_actions(state, True):
                next_state = state.next(action)
                eval = self.alpha_beta(next_state, depth + 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break  # Beta pruning
            return max_eval
        else:
            min_eval = float('inf')
            for action in self.sorted_legal_actions(state, False):
                next_state = state.next(action)
                eval = self.alpha_beta(next_state, depth + 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break  # Alpha pruning
            return min_eval

    def best_action(self): # 최적의 선택
        best_action = None
        if self.state.is_first_player(): # AI가 먼저면 최대화(승리)를 목표
            max_eval = -float('inf')
            for action in self.sorted_legal_actions(self.state, True):
                next_state = self.state.next(action)
                eval = self.alpha_beta(next_state, 0, -float('inf'), float('inf'), False)
                if eval > max_eval:
                    max_eval = eval
                    best_action = action
        else:
            min_eval = float('inf') # 상대가 먼저면 최소화(방어)를 목표
            for action in self.sorted_legal_actions(self.state, False):
                next_state = self.state.next(action)
                eval = self.alpha_beta(next_state, 0, -float('inf'), float('inf'), True)
                if eval < min_eval:
                    min_eval = eval
                    best_action = action

        return best_action

In [ ]:
import time
def play_game():
    # 게임 시작 시 사용자에게 차례 선택
    print("Do you want to go first or let the computer go first?")
    choice = input("Type '1' for you first or '-1' for computer first: ").strip()

    if choice == '1':
        state = State(turn=1)  # 사용자가 먼저
    elif choice == '-1':
        state = State(turn=-1)  # 컴퓨터가 먼저
    else:
        print("Invalid choice, defaulting to you first.")
        state = State(turn=1)
    while not state.is_done():
        state.render()
        if state.is_first_player():
            # 사용자의 차례
            print("Your turn! Choose a position (0-8): ")
            action = int(input())
            if action not in state.legal_actions():
                print("잘못된 입력입니다. 다시 선택해주세요")
                continue
        else:
            # 컴퓨터의 차례
            print("Computer's turn...")
            ai = Alphabeta(state)
            start_time = time.time()
            action = ai.best_action()
            end_time = time.time()
            print(f"Computer took {end_time - start_time:.2f} seconds to decide.")

        # 상태 업데이트
        state = state.next(action)

    # 결과 출력
    state.render()
    if state.is_win():
        print("You win!" if state.is_first_player() else "Computer wins!")
    elif state.is_lose():
        print("Computer wins!" if state.is_first_player() else "You win!")
    else:
        print("It's a draw!")

In [ ]:
play_game()

Do you want to go first or let the computer go first?
Type '1' for you first or '-1' for computer first: -1
---
---
---
Computer's turn...
Computer took 0.03 seconds to decide.
x--
---
---
Your turn! Choose a position (0-8): 
2
x-o
---
---
Computer's turn...
Computer took 0.02 seconds to decide.
x-o
---
x--
Your turn! Choose a position (0-8): 
3
x-o
o--
x--
Computer's turn...
Computer took 0.00 seconds to decide.
x-o
o-x
x--
Your turn! Choose a position (0-8): 
1
xoo
o-x
x--
Computer's turn...
Computer took 0.00 seconds to decide.
xoo
o-x
x-x
Your turn! Choose a position (0-8): 
7
xoo
o-x
xox
Computer's turn...
Computer took 0.00 seconds to decide.
xoo
oxx
xox
Computer wins!


- 1로 설정해서 내가 먼저 시작할 경우 AI는 방어적인 형태를 취하여 이길 수 있는 상황임에도 내 것을 막는 데에 집중함
- -1로 설정하여 AI가 먼저 시작할 경우 확실히 공격적인 느낌